# Detector de Texto para OCR
Este modelo pretende detectar la resencia de texto en una imagen y en lo posible retornar las lineas de texto y cajas de los carcteres que las componen.

Modelo basado en yolo 8

## Importación de Librerias

In [ ]:
#!pip install -q ultralytics
from ultralytics import YOLO

In [ ]:
#!pip install pandas numpy seaborn matplotlib scikit-learn pillow opencv-python IPython

In [ ]:
#Importo librerias a utilizar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import cv2
import random
from PIL import Image, ImageFilter
from IPython.display import Image as ImageDisplay
from IPython.core.display import HTML
from io import BytesIO
import base64
from sklearn.model_selection import train_test_split

In [ ]:
# Funciones auxiliares
def np_to_base64_img(arr):
    """Convierte un np.array a una imagen embebida en base64"""
    h,w = arr.shape[:2]
    im = Image.fromarray(arr)
    buffer = BytesIO()
    im.save(buffer, format="PNG")
    img_str = base64.b64encode(buffer.getvalue()).decode()
    return f'<img src="data:image/png;base64,{img_str}" width="{w}" height="{h}"/>'

## Dataset de entrenamiento
Este dataset contendra imagenes de carcateres y las coordenadas de la caja que los contiene

In [ ]:
def get_bbox_from_image(image_array):
    """
    Toma un array NumPy de una imagen en escala de grises, encuentra los contornos,
    los combina y devuelve las coordenadas de la bounding box en formato
    (x_min, y_min, x_max, y_max).

    Args:
        image_array (np.ndarray): Una imagen en escala de grises como un array NumPy.

    Returns:
        tuple: Una tupla (x_min, y_min, x_max, y_max) que representa la bounding box.
               Retorna None si no se encuentran contornos.
    """
    # Asegurarse de que la imagen esté en escala de grises si no lo está ya
    if len(image_array.shape) == 3:
        gray_image = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image_array

    # Binarizar la imagen si es necesario para asegurar buenos contornos
    # Esto es crucial si la imagen de entrada tiene tonos de gris en lugar de solo blanco/negro
    # Ajusta el umbral (e.g., 127) y el tipo (e.g., cv2.THRESH_BINARY_INV) según tus imágenes.
    _, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)


    # Encontrar contornos
    # cv2.RETR_EXTERNAL recupera solo los contornos externos (útil para caracteres con agujeros internos)
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        binary_image=cv2.bitwise_not(binary_image)
        contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            return None # No se encontraron contornos

    # Combinar todos los contornos en uno solo (para caracteres con múltiples partes como 'i' o 'j')
    # Solo apilamos si hay contornos válidos (size > 0)
    valid_contours = [c.squeeze() for c in contours if c.size > 0]
    if not valid_contours:
        return None # No se encontraron contornos válidos después de limpiar

    all_points = np.vstack(valid_contours)

    # Encontrar el rectángulo delimitador para el contorno combinado
    x, y, w, h = cv2.boundingRect(all_points)

    # Retornar en el formato (x_min, y_min, x_max, y_max)
    x_min, y_min = x, y
    x_max, y_max = x + w, y + h

    return (x_min, y_min, x_max, y_max)

In [ ]:
# Cargar los datos del archivo npz
data = np.load(fr"C:\Users\Andres\Downloads\unified_dataset.npz", allow_pickle=True)

print(format(data))
images = data['images']
bboxes = data['bboxes']

# Crear un DataFrame de pandas
df = pd.DataFrame({'image': list(images), 'bbox':list(bboxes)})

# Aplicar la función np_to_base64_img a la columna 'image'
df['imagen_base64'] = df['image'].apply(np_to_base64_img)
df['bbox'] = df['image'].apply(get_bbox_from_image)

# Mostrar las primeras filas del nuevo DataFrame
print(df.head())

## Funciones de transformación para aumentado de datos

In [ ]:
# --- Funciones auxiliares para transformar bounding boxes ---

def get_corners_from_bbox(bbox):
    """Convierte (x_min, y_min, x_max, y_max) a 4 puntos de esquina."""
    x_min, y_min, x_max, y_max = bbox
    return np.array([
        [x_min, y_min],
        [x_max, y_min],
        [x_max, y_max],
        [x_min, y_max]
    ], dtype=np.float32)

def get_bbox_from_corners(corners, image_shape):
    """
    Calcula la nueva bbox (x_min, y_min, x_max, y_max) a partir de 4 puntos de esquina transformados,
    asegurándose de que la bbox esté dentro de los límites de la imagen.
    """
    height, width = image_shape[:2]

    x_coords = corners[:, 0]
    y_coords = corners[:, 1]

    x_min = np.min(x_coords)
    y_min = np.min(y_coords)
    x_max = np.max(x_coords)
    y_max = np.max(y_coords)

    # Asegurarse de que las coordenadas estén dentro de los límites de la imagen
    x_min = max(0, int(x_min))
    y_min = max(0, int(y_min))
    x_max = min(width - 1, int(x_max))
    y_max = min(height - 1, int(y_max))

    # Asegurarse de que la caja sea válida (x_max >= x_min, y_max >= y_min)
    if x_max < x_min: x_max = x_min
    if y_max < y_min: y_max = y_min

    return (x_min, y_min, x_max, y_max)

# --- Funciones de Rotación, Escala y Perspectiva Modificadas ---

# Rotación
def rotar_imagen(imagen_np_array, bbox, angulo_rotacion_min, angulo_rotacion_max):
    """
    Rota una imagen y su bounding box asociada.
    La imagen de entrada debe ser un NumPy array.
    La bbox de entrada es (x_min, y_min, x_max, y_max).
    Devuelve la imagen rotada (NumPy array) y la nueva bbox.
    """
    img = Image.fromarray(imagen_np_array)
    ancho_original, alto_original = img.size

    # Aplico una rotación aleatoria dentro del rango permitido
    angulos_rotacion_opciones = [0, 0, random.uniform(angulo_rotacion_min, angulo_rotacion_max)]
    angulo_rotacion = random.choice(angulos_rotacion_opciones)

    # Rota la imagen, expandiendo el lienzo para que no se corte
    imagen_rotada_pil = img.rotate(angulo_rotacion, resample=Image.Resampling.BILINEAR, expand=True)
    imagen_rotada_np = np.array(imagen_rotada_pil)

    # Calcular la matriz de rotación para los puntos de la bbox
    # El centro de rotación cambia si expandimos la imagen
    ancho_nuevo, alto_nuevo = imagen_rotada_pil.size
    # Desplazamiento del centro (el centro de la imagen original se mueve al nuevo centro)
    dx = (ancho_nuevo - ancho_original) / 2
    dy = (alto_nuevo - alto_original) / 2

    # Matriz de rotación con traslación para el centro original
    M_rot = cv2.getRotationMatrix2D((ancho_original / 2, alto_original / 2), angulo_rotacion, 1.0)
    # Aplicar el desplazamiento para el nuevo tamaño de lienzo
    M_rot[0, 2] += dx
    M_rot[1, 2] += dy

    # Transformar las esquinas de la bbox
    bbox_corners = get_corners_from_bbox(bbox)
    # cv2.transform necesita los puntos en formato [N, 1, 2]
    transformed_corners = cv2.transform(bbox_corners.reshape(-1, 1, 2), M_rot).reshape(-1, 2)

    # Calcular la nueva bbox a partir de las esquinas transformadas
    nueva_bbox = get_bbox_from_corners(transformed_corners, imagen_rotada_np.shape)

    return imagen_rotada_np, nueva_bbox

# Perspectiva Vertical
def aplicar_perspectiva_vertical(imagen_np_array, bbox, factor, top=True):
    """
    Aplica una transformación de perspectiva vertical a una imagen y su bounding box.
    La imagen de entrada debe ser un NumPy array (BGR o escala de grises).
    La bbox de entrada es (x_min, y_min, x_max, y_max).
    Devuelve la imagen con perspectiva y la nueva bbox.
    """
    height, width = imagen_np_array.shape[:2]

    # Puntos originales de la imagen (esquinas)
    pts1_img = np.float32([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]])

    # Puntos de destino para la imagen
    pts2_img = np.copy(pts1_img)

    if 0 <= factor <= 1:
        if top:
            new_top_left_x = int(width * (1 - factor) / 2)
            new_top_right_x = width - 1 - new_top_left_x
            pts2_img[0] = [new_top_left_x, 0]
            pts2_img[1] = [new_top_right_x, 0]
        else:
            new_bottom_left_x = int(width * (1 - factor) / 2)
            new_bottom_right_x = width - 1 - new_bottom_left_x
            pts2_img[2] = [new_bottom_right_x, height - 1]
            pts2_img[3] = [new_bottom_left_x, height - 1]
    else:
        print("Advertencia: El factor de perspectiva debe estar entre 0 y 1. No se aplicó la transformación.")
        return imagen_np_array, bbox

    # Obtener la matriz de transformación de perspectiva
    M_perspectiva = cv2.getPerspectiveTransform(pts1_img, pts2_img)

    # Aplicar la transformación de perspectiva a la imagen
    imagen_con_perspectiva = cv2.warpPerspective(imagen_np_array, M_perspectiva, (width, height))

    # Transformar las esquinas de la bbox usando la misma matriz
    bbox_corners = get_corners_from_bbox(bbox)
    transformed_corners = cv2.perspectiveTransform(bbox_corners.reshape(-1, 1, 2), M_perspectiva).reshape(-1, 2)

    # Calcular la nueva bbox a partir de las esquinas transformadas
    nueva_bbox = get_bbox_from_corners(transformed_corners, imagen_con_perspectiva.shape)

    return imagen_con_perspectiva, nueva_bbox

# Perspectiva Horizontal
def aplicar_perspectiva_horizontal(imagen_np_array, bbox, factor, left=True):
    """
    Aplica una transformación de perspectiva horizontal a una imagen y su bounding box.
    La imagen de entrada debe ser un NumPy array (BGR o escala de grises).
    La bbox de entrada es (x_min, y_min, x_max, y_max).
    Devuelve la imagen con perspectiva y la nueva bbox.
    """
    height, width = imagen_np_array.shape[:2]

    # Puntos originales de la imagen (esquinas)
    pts1_img = np.float32([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]])

    # Puntos de destino para la imagen
    pts2_img = np.copy(pts1_img)

    if 0 <= factor <= 1:
        if left:
            new_top_left_y = int(height * (1 - factor) / 2)
            new_bottom_left_y = height - 1 - new_top_left_y
            pts2_img[0] = [0, new_top_left_y]
            pts2_img[3] = [0, new_bottom_left_y]
        else:
            new_top_right_y = int(height * (1 - factor) / 2)
            new_bottom_right_y = height - 1 - new_top_right_y
            pts2_img[1] = [width - 1, new_top_right_y]
            pts2_img[2] = [width - 1, new_bottom_right_y]
    else:
        print("Advertencia: El factor de perspectiva debe estar entre 0 y 1. No se aplicó la transformación.")
        return imagen_np_array, bbox

    # Obtener la matriz de transformación de perspectiva
    M_perspectiva = cv2.getPerspectiveTransform(pts1_img, pts2_img)

    # Aplicar la transformación de perspectiva a la imagen
    imagen_con_perspectiva = cv2.warpPerspective(imagen_np_array, M_perspectiva, (width, height))

    # Transformar las esquinas de la bbox usando la misma matriz
    bbox_corners = get_corners_from_bbox(bbox)
    transformed_corners = cv2.perspectiveTransform(bbox_corners.reshape(-1, 1, 2), M_perspectiva).reshape(-1, 2)

    # Calcular la nueva bbox a partir de las esquinas transformadas
    nueva_bbox = get_bbox_from_corners(transformed_corners, imagen_con_perspectiva.shape)

    return imagen_con_perspectiva, nueva_bbox


In [ ]:
# prompt: quiero una funcion que rescale la imagen a 32x32 si fuera mas grande la imagen y ajuste el bbox a ese tamaño en ese caso

def rescale_image_and_bbox(image_np_array, bbox, target_size=(32, 32)):
    """
    Rescala una imagen a un tamaño objetivo si es más grande y ajusta la bounding box.

    Args:
        image_np_array (np.ndarray): La imagen como un array NumPy.
        bbox (tuple): La bounding box en formato (x_min, y_min, x_max, y_max).
        target_size (tuple): El tamaño objetivo para la imagen (ancho, alto).

    Returns:
        tuple: Una tupla (rescaled_image, rescaled_bbox).
               retorna la imagen y bbox originales si la imagen no es más grande que target_size.
               retorna None, None si la imagen no tiene una bbox válida.
    """

    current_height, current_width = image_np_array.shape[:2]
    target_width, target_height = target_size

    # Solo rescalar si la imagen es más grande que el tamaño objetivo en alguna dimensión
    if current_width > target_width or current_height > target_height:
        # Calcular el factor de escala
        scale_w = target_width / current_width
        scale_h = target_height / current_height

        # Rescalar la imagen usando cv2.resize
        rescaled_image = cv2.resize(image_np_array, target_size, interpolation=cv2.INTER_AREA)

        # Ajustar la bounding box
        x_min, y_min, x_max, y_max = bbox
        rescaled_bbox = (
            int(x_min * scale_w),
            int(y_min * scale_h),
            int(x_max * scale_w),
            int(y_max * scale_h)
        )

        return rescaled_image, rescaled_bbox
    else:
        # No necesita rescalar, retornar original
        return image_np_array, bbox



In [ ]:
#Funciones de ruido
# Ruido de Sal
def add_salt_and_pepper_noise(image, prob):
    """Agrega ruido de sal y pimienta a una imagen PIL."""
    img_array = np.array(image)
    output = np.copy(img_array)
    black = 0
    white = 255
    height, width, channels = img_array.shape if len(img_array.shape) == 3 else (img_array.shape[0], img_array.shape[1], 1)

    for i in range(height):
        for j in range(width):
            if random.random() < prob:
                if random.random() < 0.5:
                    output[i][j] = black
                else:
                    output[i][j] = white
    return output

# Desenfoque
def add_gaussian_blur(imagen_gris_cv2, radius):
    """Aplica desenfoque gaussiano a una imagen CV2 en escala de grises."""
    # cv2.GaussianBlur espera un tamaño de kernel impar y positivo
    # Asegurarse de que el tamaño del kernel sea impar y al menos 1
    kernel_size = int(radius * 2) + 1
    if kernel_size % 2 == 0: # Si es par, hacerlo impar
        kernel_size += 1
    kernel_size = max(1, kernel_size) # Asegurarse de que sea al menos 1
    return cv2.GaussianBlur(imagen_gris_cv2, (kernel_size, kernel_size), 0)

def add_motion_blur(imagen_gris_cv2, radius, angle):
    """Aplica desenfoque de movimiento a una imagen CV2 en escala de grises."""
    longitud = int(radius)
    if longitud == 0:
        return imagen_gris_cv2

    kernel = np.zeros((longitud, longitud), dtype=np.float32)
    center = longitud // 2
    # Crear una línea horizontal para simular el movimiento
    cv2.line(kernel, (0, center), (longitud - 1, center), 1, 1)

    # Rotar el kernel
    M = cv2.getRotationMatrix2D((center, center), angle, 1.0)
    kernel = cv2.warpAffine(kernel, M, (longitud, longitud), flags=cv2.INTER_LINEAR)
    kernel = kernel / (np.sum(kernel) + 1e-6) # Normalizar y evitar división por cero

    output = cv2.filter2D(imagen_gris_cv2, -1, kernel)
    return output

In [ ]:
# Funciones de variacion del color
def ajustar_brillo(imagen_gris_cv2, valor_brillo):
    """Ajusta el brillo de una imagen CV2 en escala de grises."""
    # Usando np.clip con operaciones aritméticas directas de NumPy para mayor robustez.
    return np.clip(imagen_gris_cv2.astype(np.int16) + valor_brillo, 0, 255).astype(np.uint8)

def ajustar_contraste(imagen_gris, factor_contraste):
    # Asegura que el factor de contraste sea positivo
    factor_clipeado = max(0.0, factor_contraste)

    # Aplica el ajuste de contraste
    # Convertir a float para evitar problemas con la multiplicación y luego a uint8
    imagen_float = imagen_gris.astype(np.float32)

    # Fórmula: alpha * pixel_value + beta (donde beta = 128 * (1 - alpha) para mantener el punto medio)
    # Sin embargo, la forma más directa es como la fórmula mencionada: alpha * (P - medio) + medio
    # OpenCV's convertScaleAbs hace (alpha * src + beta)
    # Para la fórmula de contraste que dimos, beta sería: 128 * (1 - factor_contraste)

    imagen_ajustada = cv2.convertScaleAbs(imagen_float, alpha=factor_clipeado, beta=128 * (1 - factor_clipeado))

    return imagen_ajustada

In [ ]:
def aplicar_transformacion_aleatoria(imagen_cv2, bbox):
    """
    Decide si aplicar transformaciones (50% de probabilidad) y, si lo hace,
    aplica entre 1 y 3 transformaciones aleatorias.
    Todas las transformaciones reciben y devuelven imágenes CV2 en escala de grises.

    Args:
        imagen_cv2 (np.ndarray): La imagen de entrada en formato numpy array (BGR o escala de grises).

    Returns:
        np.ndarray: La imagen transformada o la original, siempre en escala de grises (CV2).
    """
    # Primero, asegurar que la imagen de entrada esté en escala de grises
    if len(imagen_cv2.shape) == 3:
        imagen_gris_original = cv2.cvtColor(imagen_cv2, cv2.COLOR_BGR2GRAY)
    else:
        imagen_gris_original = imagen_cv2.copy() # Copia para no modificar la original

    # 50% de probabilidad de no aplicar ninguna transformación
    #if random.random() < 0.5:
    #    return imagen_gris_original, bbox # Devuelve la imagen original en escala de grises y la caja original

    # Si llegamos aquí, vamos a aplicar transformaciones
    imagen_actual = imagen_gris_original # Empezamos con la versión gris de la imagen

    # Lista de transformaciones disponibles (TODAS operan en CV2 gris y devuelven CV2 gris)
    transformaciones_disponibles = [
        # Las funciones deben ser las versiones que reciben/devuelven CV2 gris
        {'func': add_salt_and_pepper_noise, 'params': {'prob': random.uniform(0.005, 0.03)}},
        {'func': add_gaussian_blur, 'params': {'radius': random.uniform(0.5, 1.5)}},
        {'func': add_motion_blur, 'params': {'radius': random.randint(5, 15), 'angle': random.uniform(0, 360)}},
        {'func': rotar_imagen, 'params': {'bbox': bbox, 'angulo_rotacion_min': -5, 'angulo_rotacion_max': 5}},
        {'func': aplicar_perspectiva_vertical, 'params': {'bbox': bbox, 'factor': random.uniform(0.05, 0.2), 'top': random.choice([True, False])}},
        {'func': aplicar_perspectiva_horizontal, 'params': {'bbox': bbox, 'factor': random.uniform(0.05, 0.2), 'left': random.choice([True, False])}},
        {'func': ajustar_brillo, 'params': {'valor_brillo': random.randint(-30, 30)}},
        {'func': ajustar_contraste, 'params': {'factor_contraste': random.uniform(0.8, 1.2)}},
    ]

    # Elegir entre 1 y 3 transformaciones para aplicar
    num_transformaciones = random.randint(1, 3)
    # Usamos random.sample para elegir funciones únicas si num_transformaciones es menor que la lista completa
    transformaciones_a_aplicar = random.sample(transformaciones_disponibles, num_transformaciones)

    for trans_info in transformaciones_a_aplicar:
        func = trans_info['func']
        params = trans_info['params']

        try:
            # Todas las funciones ahora reciben y devuelven CV2 en escala de grises
            resultado = func(imagen_actual, **params)
            if isinstance(resultado, tuple) and len(resultado) == 2:
                imagen_actual, bbox = resultado
            else:
                imagen_actual = resultado
        except Exception as e:
            print(f"Error al aplicar la transformación '{func.__name__}': {e}. Se mantiene la imagen actual para la siguiente transformación.")

    return imagen_actual, bbox

## Generacion de archivos de entrenamiento

In [ ]:
os.makedirs('c:/tmp/content/training/images/train', exist_ok=True)
os.makedirs('c:/tmp/content/training/images/val', exist_ok=True)
os.makedirs('c:/tmp/content/training/labels/train', exist_ok=True)
os.makedirs('c:/tmp/content/training/labels/val', exist_ok=True)

indices = df.index.to_list()
train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42)

def save_image_and_label(idx, image, bbox, split):
    img_path = f'c:/tmp/content/training/images/{split}/{idx}.jpg'
    label_path = f'c:/tmp/content/training/labels/{split}/{idx}.txt'

    img = Image.fromarray((image * 255).astype('uint8')).convert('L')
    img.save(img_path)

    h, w = image.shape
    x_min, y_min, x_max, y_max = bbox

    # YOLO format: class x_center y_center width height (normalized)
    xc = (x_min + x_max) / 2 / w
    yc = (y_min + y_max) / 2 / h
    bw = (x_max - x_min) / w
    bh = (y_max - y_min) / h

    with open(label_path, 'w') as f:
        f.write(f'0 {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n')

for idx, row in df.iterrows():
    split = 'train' if idx in train_idx else 'val'
    image, bbox = rescale_image_and_bbox(row['image'], row['bbox'])
    save_image_and_label(idx, image, bbox, split)

In [ ]:
with open("data.yaml", "w") as f:
    f.write("""
path: c:/tmp/content/training
train: images/train
val: images/val
names: ['char']
""")

## Entrenamiento del Modelo YOLO 8

In [ ]:
model = YOLO('yolov8n.pt')  # También podés usar yolov8s.pt para más precisión

model.train(
    data='data.yaml',
    epochs=20,
    imgsz=32,
    batch=64,
    show=True,
    augment=True
)